Plot how the currents change with time
====
Might be interesting

In [ ]:
import pathlib

data_path = pathlib.Path(
    "~/geog_rdsf/data/projects/dtop/cmip5/historical/cmip5_historical_mdts_yr5.dat"
).expanduser()
mdata_path = data_path.with_name(data_path.stem + "_meta.txt")

assert mdata_path.exists()
assert data_path.exists()

In [ ]:
from current_denoising.generation import ioutils

model = "NorESM1-M"
name = "r2i1p1"
year = 1861

data = ioutils.read_clean_currents(
    data_path, mdata_path, year=year, model=model, name=name
)

In [ ]:
from current_denoising.plotting import maps

maps.imshow(data)

Animate how the currents change by year...
----

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Years from 1851 to 2001
# CMIP6 also has 2006, but CMIP5 doesn't
years = np.arange(1851, 2006, 5)

fig, ax = plt.subplots(figsize=(10, 5))

vmin, vmax = np.quantile(data[np.isfinite(data)], [0.01, 0.99])
im = ax.imshow(data, vmin=vmin, vmax=vmax, cmap="turbo")

fig.colorbar(im)


def update(year):
    data = ioutils.read_clean_currents(
        data_path,
        mdata_path,
        year=year,
        model=model,
        name=name,
    )

    im.set_array(data)
    ax.set_title(f"Year: {year} Model: {model}")
    return [im]


ani = FuncAnimation(fig, update, frames=tqdm(years), blit=True)

In [ ]:
ani.save(
    "changing_currents.mp4",
    writer="ffmpeg",
    fps=6,
    bitrate=2000,
    extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p", "-preset", "fast"],
)